In [2]:
# Dependency
from Bio import Entrez
import pytaxonkit as ptax
import pandas as pd
import time
import os

In [3]:
#Input
taxids=[int(i) for i in set(pd.read_csv(snakemake.input[0],sep="\t")["Taxa"])]

In [4]:
#Get taxnames
species_names=ptax.name(taxids,data_dir=os.path.dirname(snakemake.input[1]))["Name"]

In [5]:
os.mkdir(snakemake.output[0])
#Register
Entrez.email = snakemake.params[0]
#Output
#approach correct result
for index,species_name in enumerate(species_names):
    start = time.time()
    handle=Entrez.esearch(db="nucleotide", retmax=10, term=species_name+" [Organism]")
    records=Entrez.read(handle)
    handle.close()
    best_match=records["IdList"][0]
    handle=Entrez.esummary(db="nucleotide",id=best_match)
    record=Entrez.read(handle)
    handle.close()
    #TODO put safety net in place
    #assert(species_name.upper().strip() in record[0]['Title'].upper().strip())
    handle=Entrez.efetch(db="nucleotide",id=best_match,rettype="fasta_cds_na",retmode="text")
    out_fasta=handle.read()
    handle.close()
    time.sleep(1)
    #Output
    with open(snakemake.output[0]+"/"+str(taxids[index])+"_mrna.fa",'w') as outfile:
        print(out_fasta,file=outfile)
    time.sleep(max(0,1-(time.time()-start))) #Only 3 request/seconds are allowed...